Basic topic modelling for just the contexts and their topics


- right now running it without removing non-words!

Both global_tuning and evolutionary_tuning are set to True as a default, but can easily be changed. Perhaps you do not want the representations to be influenced by the global representation and merely see how they evolved over time:


to do if i care
- remove stopwords
- remove non-words somehow (not worth the computational effort)
- maybe make the fact/fiction also into periods
- MAKE MORE EFFICIENT!!!
- maybe subset the topics for fact/fiction to be to 90s and after 90s
- frequency with removed stop.words is fucky

FUNCTIONS ETC

In [18]:
import pandas as pd
from pathlib import Path
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from concurrent.futures import ThreadPoolExecutor, as_completed

stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

def process_file(txt_file, metadata_df):
    filename = txt_file.name
    file_date = metadata_df.loc[metadata_df['filename'] == filename.replace('.txt', '.pdf'), 'date'].values
    if file_date.size > 0:
        try:
            period = pd.Period(pd.to_datetime(file_date[0], format='%Y/%m/%d').strftime('%Y'), freq='Y')
            with open(txt_file, 'r', encoding='utf-8') as f:
                text = f.read()
            return text, period
        except (pd.errors.OutOfBoundsDatetime, ValueError):
            print(f"Invalid date for file: {filename}")
            return None, None
    else:
        print(f"Date not found for file: {filename}")
        return None, None

def load_texts_and_dates(folder_path, metadata_df):
    texts = []
    periods = []
    
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_file, txt_file, metadata_df): txt_file for txt_file in folder_path.glob('*.txt')}
        for future in as_completed(futures):
            text, period = future.result()
            if text is not None and period is not None:
                texts.append(text)
                periods.append(period)
    
    return texts, periods



In [19]:
def process_texts(folders):
    all_texts = []
    all_periods = []
    
    for key, (folder_path, metadata_path) in folders.items():
        metadata_df = load_metadata(metadata_path)
        texts, periods = load_texts_and_dates(folder_path, metadata_df)
        all_texts.extend(texts)
        all_periods.extend(periods)
        print(f"Processed {len(texts)} files from {folder_path}")
    
    period_strings = [period.strftime('%Y-%m-%d') for period in all_periods]
    processed_texts = [remove_stop_words(text) for text in all_texts]
    
    return processed_texts, period_strings

b_folders = {
    'rstb': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstb'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstb.csv'),
}

l_folders = {
    'rstb': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstl'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstl.csv'),
}

a_folders = {
    'rstb': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rsta'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rsta.csv'),
}

b_texts, b_periods = process_texts(b_folders)
l_texts, l_periods = process_texts(l_folders)
a_texts, a_periods = process_texts(a_folders)

Processed 5284 files from D:\Fact_fiction_corpus\texts\royal society\txt_rstb


FACT/FICTION CONTEXTS

In [ ]:
import pandas as pd
from pathlib import Path
from bertopic import BERTopic

#contexts-only (kind of long though)
rs_data = pd.concat([pd.read_excel("../data_for_viewing/extracted_raw/ff_royal_society_words_found_full_RSTB.xlsx"), pd.read_excel("../data_for_viewing/extracted_raw/ff_royal_society_words_found_full_RSTL.xlsx"), pd.read_excel("../data_for_viewing/extracted_raw/ff_royal_society_words_found_full_RSTA.xlsx")], ignore_index=True)
rs_data['Date'] = pd.to_datetime(rs_data['Date'], errors='coerce')
rs_data = rs_data.dropna(subset=['Date'])

# Fact
fact_data = rs_data[rs_data['Keyword'].str.contains('act', case=False, na=False)]
date_fact = fact_data['Date'].tolist()
contexts_fact = fact_data['Context'].to_list()

# Fiction
fiction_data = rs_data[rs_data['Keyword'].str.contains('ict', case=False, na=False)]
date_fict = fiction_data['Date'].tolist()
contexts_fict = fiction_data['Context'].to_list()

In [ ]:
#fact/fict with no stop words
fict_data_clean = [remove_stop_words(text) for text in contexts_fict]
fact_data_clean = [remove_stop_words(text) for text in contexts_fact]

topic_model = BERTopic(verbose=False)
topics, probs = topic_model.fit_transform(fact_data_clean)
topics_over_time = topic_model.topics_over_time(fact_data_clean, date_fact, datetime_format="%Y-%m-%d", nr_bins=30)
fig1 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title= "Topics over time in Fact words found in Royal Society Transactions (RSTL, RSTB, RSTA)")
fig1.write_html('fact_topics_over_time_30_nostopwords.html')

fig1.show()

topic_model = BERTopic(verbose=False)
topics, probs = topic_model.fit_transform(fict_data_clean)
topics_over_time = topic_model.topics_over_time(fict_data_clean, date_fict, datetime_format="%Y-%m-%d", nr_bins=30)
fig2 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title= "Topics over time in Fiction words found in Royal Society Transactions (RSTL, RSTB, RSTA)")
fig2.write_html('fiction_topics_over_time_30_nostopwords.html')

fig2.show()

In [ ]:
#with stopwords

# Fact (all together!, rstl, rstb, rsta) #this uses rs_data and not the full data
topic_model = BERTopic(verbose=False)
topics, probs = topic_model.fit_transform(contexts_fact)
topics_over_time = topic_model.topics_over_time(contexts_fact, date_fact, datetime_format="%Y-%m-%d", nr_bins=30)
fig11 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title= "Topics over time in Fact words found in Royal Society Transactions (RSTL, RSTB, RSTA)")
fig11.write_html('fact_topics_over_time_30.html')

fig11.show()

# Fiction (all together!, rstl, rstb, rsta) #this uses rs_data and not the full data
topic_model = BERTopic(verbose=False)
topics, probs = topic_model.fit_transform(contexts_fict)
topics_over_time = topic_model.topics_over_time(contexts_fict, date_fict, datetime_format="%Y-%m-%d", nr_bins=30)
fig22 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title= "Topics over time in Fiction words found in Royal Society Transactions (RSTL, RSTB, RSTA)")
fig22.write_html('fiction_topics_over_time_30.html')

fig22.show()

THE TOPIC OF ALL FULL THE ROYAL SOCIETY ARTICLES?

In [ ]:
all_texts_combined = b_texts + l_texts + a_texts
all_periods_combined = b_periods + l_periods + a_periods
period_strings_combined = [period for period in all_periods_combined]

topic_model = BERTopic(verbose=False)
topics, probs = topic_model.fit_transform(all_texts_combined)

topics_over_time = topic_model.topics_over_time(all_texts_combined, period_strings_combined, datetime_format="%Y-%m-%d", nr_bins=30)
fig4 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title="Topics over time in Fact and Fiction texts (RSTL, RSTB, RSTA)")

fig4.write_html('full_rs_topics_over_time_periods_30.html')
fig4.show()

In [14]:
#periods rstl rstb

import pandas as pd
from pathlib import Path
from bertopic import BERTopic

folders = {
    'rstb': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstb'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstb.csv'),
    'rstl': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstl'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstl.csv'),
}

all_texts = []
all_periods = []
for key, (folder_path, metadata_path) in folders.items():
    metadata_df = load_metadata(metadata_path)
    texts, periods = load_texts_and_dates(folder_path, metadata_df)
    all_texts.extend(texts)
    all_periods.extend(periods)

period_strings = [period.strftime('%Y-%m-%d') for period in all_periods]

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(all_texts)

topics_over_time = topic_model.topics_over_time(all_texts, period_strings, datetime_format="%Y-%m-%d", nr_bins=30)
fig5 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title="Topics over time in Fact and Fiction texts (RSTL, RSTB)")
fig5.write_html('rstbl_topics_over_time_periods_30.html')

fig5.show()

Invalid date for file: rstl_1665_0143.txt
Invalid date for file: rstl_1665_0147.txt
Invalid date for file: rstl_1665_0148.txt
Invalid date for file: rstl_1665_0149.txt
Invalid date for file: rstl_1665_0154.txt
Invalid date for file: rstl_1665_0153.txt
Invalid date for file: rstl_1665_0152.txt
Invalid date for file: rstl_1665_0151.txt
Invalid date for file: rstl_1665_0150.txt
Invalid date for file: rstl_1666_0001.txt
Invalid date for file: rstl_1666_0002.txt
Invalid date for file: rstl_1666_0003.txt
Invalid date for file: rstl_1666_0005.txt
Invalid date for file: rstl_1666_0004.txt
Invalid date for file: rstl_1666_0006.txt
Invalid date for file: rstl_1666_0007.txt
Invalid date for file: rstl_1666_0008.txt
Invalid date for file: rstl_1666_0009.txt
Invalid date for file: rstl_1666_0010.txt
Invalid date for file: rstl_1666_0011.txt
Invalid date for file: rstl_1666_0012.txt
Invalid date for file: rstl_1666_0013.txt
Invalid date for file: rstl_1666_0014.txt
Invalid date for file: rstl_1666_0

2024-09-18 16:23:35,595 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/405 [00:00<?, ?it/s]

2024-09-18 16:27:08,561 - BERTopic - Embedding - Completed ✓
2024-09-18 16:27:08,562 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-18 16:27:14,641 - BERTopic - Dimensionality - Completed ✓
2024-09-18 16:27:14,643 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-18 16:27:15,062 - BERTopic - Cluster - Completed ✓
2024-09-18 16:27:15,068 - BERTopic - Representation - Extracting topics from clusters using representation models.


KeyboardInterrupt: 

In [8]:
#periods rsta

import pandas as pd
from pathlib import Path
from bertopic import BERTopic

folders = {
    'rsta': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rsta'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rsta.csv')
}

all_texts = []
all_periods = []
for key, (folder_path, metadata_path) in folders.items():
    metadata_df = load_metadata(metadata_path)
    texts, periods = load_texts_and_dates(folder_path, metadata_df)
    all_texts.extend(texts)
    all_periods.extend(periods)

period_strings = [period.strftime('%Y-%m-%d') for period in all_periods]

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(all_texts)

topics_over_time = topic_model.topics_over_time(all_texts, period_strings, datetime_format="%Y-%m-%d", nr_bins=30)
fig6 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title="Topics over time in Fact and Fiction texts (RSTA)")
fig6.write_html('rsta_topics_over_time_periods_30.html')

fig6.show()

Processed file: rsta20230326.txt
Processed file: rsta_2022_0155.txt
Processed file: rsta20230324.txt
Processed file: rsta_2022_0143.txt
Processed file: rsta_2015_0174.txt
Processed file: rsta_2015_0190.txt
Processed file: rsta_2015_0202.txt
Processed file: rsta20230313.txt
Processed file: rsta_2015_0189.txt
Processed file: rsta_2022_0149.txt
Processed file: rsta_2015_0284.txt
Processed file: rsta_2015_0181.txt
Processed file: rsta_2015_0205.txt
Processed file: rsta_2015_0166.txt
Processed file: rsta_2016_0075.txt
Processed file: rsta_2015_0237.txt
Processed file: rsta_2013_0406.txt
Processed file: rsta_2015_0204.txt
Processed file: rsta_2022_0156.txt
Processed file: rsta_2015_0183.txt
Processed file: rsta_2015_0341.txt
Processed file: rsta_2015_0269.txt
Processed file: rsta_2016_0018.txt
Processed file: rsta_2015_0137.txt
Processed file: rsta_2015_0277.txt
Processed file: rsta_2015_0270.txt
Processed file: rsta_2015_0185.txt
Processed file: rsta_2015_0139.txt
Processed file: rsta_2013_

2024-09-18 15:27:20,017 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/138 [00:00<?, ?it/s]

2024-09-18 15:30:12,280 - BERTopic - Embedding - Completed ✓
2024-09-18 15:30:12,282 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-18 15:30:17,659 - BERTopic - Dimensionality - Completed ✓
2024-09-18 15:30:17,660 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-18 15:30:17,909 - BERTopic - Cluster - Completed ✓
2024-09-18 15:30:17,925 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-18 15:30:58,140 - BERTopic - Representation - Completed ✓
28it [03:11,  6.84s/it]
